In [60]:
from constants.constants import Chain
from database.mongodb import MongoDB
import pandas as pd
start_timestamp = 1688169600
end_timestamp = 1696118400
timestamps = [t for t in range(start_timestamp, end_timestamp, 86400)]

In [61]:
def cal_round_timestamp(timestamp):
    return int(int(timestamp)/86400)*86400
    
def count_event(data, timestamp, amount, address):
    round_timestamp = cal_round_timestamp(timestamp)
    if round_timestamp not in data:
        return
    data[round_timestamp]["number_event"] += 1
    data[round_timestamp]["amount_in_usd"]+=amount
    if address not in data[round_timestamp]["wallets"]:
        data[round_timestamp]["wallets"].append(address)
            
def count_wallets(data):
    for key, value in data.items():
        value['number_wallets'] = len(list(set(value['wallets'])))
        
def check_token_liquidate_amount(tokens, timestamp, amount, token, debt=False):
    round_timestamp = cal_round_timestamp(timestamp)
    if round_timestamp not in tokens:
        return
    if debt:
        if token not in tokens[round_timestamp]["debt"]:
            tokens[round_timestamp]["debt"][token] = 0
        tokens[round_timestamp]["debt"][token] += amount
    else:
        if token not in tokens[round_timestamp]["collateral"]:
            tokens[round_timestamp]["collateral"][token] = 0
        tokens[round_timestamp]["collateral"][token] += amount
        
def check_token_liquidate_number(tokens, timestamp, amount, token, debt=False):
    round_timestamp = cal_round_timestamp(timestamp)
    if round_timestamp not in tokens:
        return
    if debt:
        if token not in tokens[round_timestamp]["debt"]:
            tokens[round_timestamp]["debt"][token] = 0
        tokens[round_timestamp]["debt"][token] += 1
    else:
        if token not in tokens[round_timestamp]["collateral"]:
            tokens[round_timestamp]["collateral"][token] = 0
        tokens[round_timestamp]["collateral"][token] += 1
    
def liquidate_analyze(data):
    df = {'timestamp':[], 'number_event':[], 'amount_in_usd': [], 'wallets':[], 'number_wallets': []}
    for key, value in data.items():
        df['timestamp'].append(key)
        df['number_event'].append(value['number_event'])
        df['amount_in_usd'].append(value['amount_in_usd'])
        df['wallets'].append(value['wallets'])
        df['number_wallets'].append(value['number_wallets'])
    df = pd.DataFrame(df)
    return df

In [72]:
liquidate_data, events, users, remove_wallets, chaintokens, tokens = {}, {}, {}, {}, {}, {}
min_tx, max_tx = 0, 0
many_liquidated_users = []
rmtimestamps = {
"ethereum":[1692230400, 1692316800, 1694390400],
"polygon": [1689206400,1690675200,1692057600,1692144000,1692230400,1692662400,1694304000,1694390400, 1695772800],
"arbitrum": [1692230400,1692316800,1692403200,1694390400,1695168000],
"avalanche":[1692230400,1692662400,1694390400]
}
for key in Chain.mapping:
    liquidate_data[key] = {t:{"number_event":0, 'amount_in_usd':0, 'wallets':[]} for t in timestamps}
    events[key] = []
    users[key] = {}
    remove_wallets[key] = []
    mongo = MongoDB("mongodb://localhost:27017/", database="blockchain_etl", db_prefix=key)
    for debtor in mongo.get_documents("debtors", {}):
        address = debtor.get('_id')
        for buyer, value in debtor.get("buyers").items():
            for timestamp, event in value.items():
                if key in ["ethereum", "polygon", "arbitrum","avalanche"] and cal_round_timestamp(timestamp) in rmtimestamps[key]:
                    remove_wallets[key].append(address)
                amount = event.get("debtAssetInUSD")
                if amount > 100000 or amount < 10:
                    remove_wallets[key].append(address)
                    if amount > 100000: max_tx += 1
                    if amount < 10: min_tx += 1
                count_event(liquidate_data[key], timestamp, amount, address)
                events[key].append(amount)
                if address not in users[key]:
                    users[key][address] = 0
                users[key][address] += 1
    
    count_wallets(liquidate_data[key])

In [5]:
liquidate_data

{'bsc': {1688169600: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688256000: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688342400: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688428800: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688515200: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688601600: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688688000: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688774400: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688860800: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets': 0},
  1688947200: {'number_event': 0,
   'amount_in_usd': 0,
   'wallets': [],
   'number_wallets

In [6]:
chaintokens = {}
for key in Chain.mapping:
    chaintokens[key]={
        1692230400:{
            "debt":{},
            "collateral":{}
        },
        1694390400:{
            "debt":{},
            "collateral":{}
        }
        }
    tokens[key]={
        1692230400:{
            "debt":{},
            "collateral":{}
        },
        1694390400:{
            "debt":{},
            "collateral":{}
        }
        }
    mongo = MongoDB("mongodb://localhost:27017/", database="blockchain_etl", db_prefix=key)
    for debtor in mongo.get_documents("debtors", {}):
        for buyer, value in debtor.get("buyers").items():
            for timestamp, event in value.items():
                token = event.get('debtAsset')
                amount = event.get('debtAssetInUSD')
                check_token_liquidate_amount(chaintokens[key], timestamp, amount, token, True)
                check_token_liquidate_number(tokens[key], timestamp, amount, token, True)
    for buyer in mongo.get_documents("liquidators", {}):
        for debtor, value in buyer.get("debtors").items():
            for timestamp, event in value.items():
                token = event.get('collateralAsset')
                amount = event.get('collateralAssetInUSD')
                check_token_liquidate_amount(chaintokens[key], timestamp, amount, token, False)
                check_token_liquidate_number(tokens[key], timestamp, amount, token, False)

## ETHEREUM

In [8]:
liquidate_analyze(liquidate_data["ethereum"]).describe()

,timestamp,number_event,amount_in_usd,number_wallets
count,9.200000e+01,92.000000,9.200000e+01,92.000000
mean,1.692101e+09,9.510870,2.684884e+05,7.663043
std,2.307058e+06,21.982758,1.642276e+06,17.030211
min,1.688170e+09,0.000000,0.000000e+00,0.000000
25%,1.690135e+09,2.000000,8.383999e+02,1.000000
50%,1.692101e+09,4.000000,4.998250e+03,4.000000
75%,1.694066e+09,8.000000,2.183112e+04,7.000000
max,1.696032e+09,144.000000,1.466141e+07,124.000000


In [9]:
event = pd.DataFrame({"events":events["ethereum"]})
event.describe()

,events
count,8.750000e+02
mean,2.822964e+04
std,2.474569e+05
min,2.358021e+00
25%,2.613240e+02
50%,1.055014e+03
75%,6.871218e+03
max,5.287871e+06


In [10]:

user = pd.DataFrame({"users":users["ethereum"].keys(), "value": users["ethereum"].values()})
user.describe()

,value
count,441.000000
mean,1.984127
std,5.566313
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,113.000000


In [67]:
# ngay nhieu liquidate
for timestamp, value in liquidate_data["ethereum"].items():
    if value.get("number_wallets") > 30:
        print(f"{timestamp} - {value.get('number_wallets')}")


1692230400 - 124
1692316800 - 33
1694390400 - 105


In [14]:
# vi bi liquidate nhieu lan
for user, value in users["ethereum"].items():
    if value > 5:
        if user not in remove_wallets["ethereum"]:
            remove_wallets["ethereum"].append(user)
        many_liquidated_users.append(user)
for timestamp in [1692230400, 1694390400]:
    remove_wallets["ethereum"]+=liquidate_data["ethereum"][timestamp]["wallets"]
remove_wallets["ethereum"] = list(set(remove_wallets["ethereum"]))

In [15]:
# sort debt token amount
m = chaintokens["ethereum"][1694390400]['debt']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2': 184.63583193224528,
 '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c': 276.7180865984761,
 '0x57ab1ec28d129707052df4df418d58a2d46d5f51': 334.53020115316167,
 '0x853d955acef822db058eb8505911ed77f175b99e': 710.534767387939,
 '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599': 3466.5937084323527,
 '0x056fd409e1d7a124bd7017459dfea2f387b6d5cd': 3888.6799999999985,
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 4408.950939024056,
 '0x4fabb145d64652a948d72533023f6e7a623c7c53': 14629.37106622157,
 '0x6b175474e89094c44da98b954eedeac495271d0f': 181317.14140812255,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 215116.68449936726,
 '0xdac17f958d2ee523a2206206994597c13d831ec7': 5411163.541152633}

In [16]:
# sort debt token amount
m = tokens["ethereum"][1694390400]['debt']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x57ab1ec28d129707052df4df418d58a2d46d5f51': 1,
 '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599': 1,
 '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c': 1,
 '0x853d955acef822db058eb8505911ed77f175b99e': 1,
 '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2': 1,
 '0x056fd409e1d7a124bd7017459dfea2f387b6d5cd': 2,
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 3,
 '0x4fabb145d64652a948d72533023f6e7a623c7c53': 10,
 '0x6b175474e89094c44da98b954eedeac495271d0f': 23,
 '0xdac17f958d2ee523a2206206994597c13d831ec7': 26,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 39}

In [17]:
# sort collateral token amount
m = chaintokens["ethereum"][1694390400]['collateral']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x8798249c2e607446efb7ad49ec89dd1865ff4272': 583.8454068614872,
 '0x0d8775f648430679a709e98d2b0cb6250d2887ef': 633.3630655821806,
 '0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2': 753.3777646245061,
 '0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0': 846.0502133770326,
 '0x111111111117dc0aa78b770fa6a738034120c302': 983.0733875427392,
 '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c': 1330.7381642764985,
 '0xba100000625a3754423978a60c9317c58a424e3d': 1893.8757239972554,
 '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984': 3432.7640953865407,
 '0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e': 3668.688312505969,
 '0xc18360217d8f7ab5e7c516566761ea12ce7f9d72': 4315.227376841829,
 '0x514910771af9ca656af840dff83e8264ecf986ca': 4699.492182234431,
 '0x0f5d2fb29fb7d3cfee444a200298f468908cc942': 10459.416374590013,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 11660.15914118916,
 '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9': 13658.216176683078,
 '0xae7ab96520de3a18e5e111b5eaab095312d7fe84': 15820.340239797633,
 '0x

In [18]:
# sort collateral token amount
m = tokens["ethereum"][1694390400]['collateral']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2': 1,
 '0x0d8775f648430679a709e98d2b0cb6250d2887ef': 1,
 '0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0': 1,
 '0x111111111117dc0aa78b770fa6a738034120c302': 2,
 '0xba100000625a3754423978a60c9317c58a424e3d': 2,
 '0x8798249c2e607446efb7ad49ec89dd1865ff4272': 2,
 '0x514910771af9ca656af840dff83e8264ecf986ca': 3,
 '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9': 3,
 '0xae7ab96520de3a18e5e111b5eaab095312d7fe84': 3,
 '0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e': 3,
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 4,
 '0xc18360217d8f7ab5e7c516566761ea12ce7f9d72': 4,
 '0xf629cbd94d3791c9250152bd8dfbdf380e2a3b9c': 5,
 '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984': 5,
 '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599': 8,
 '0x0f5d2fb29fb7d3cfee444a200298f468908cc942': 9,
 '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 52}

## POLYGON

In [20]:
liquidate_analyze(liquidate_data["polygon"]).describe()

,timestamp,number_event,amount_in_usd,number_wallets
count,9.200000e+01,92.000000,9.200000e+01,92.000000
mean,1.692101e+09,15.119565,3.205799e+04,12.750000
std,2.307058e+06,43.076432,1.563448e+05,35.419813
min,1.688170e+09,0.000000,0.000000e+00,0.000000
25%,1.690135e+09,2.000000,5.673566e+01,2.000000
50%,1.692101e+09,4.000000,5.061634e+02,4.000000
75%,1.694066e+09,9.000000,4.151082e+03,8.250000
max,1.696032e+09,354.000000,1.309529e+06,286.000000


In [21]:
event = pd.DataFrame({"events":events["polygon"]})
event.describe()

,events
count,1.391000e+03
mean,2.120298e+03
std,1.155792e+04
min,1.078000e-18
25%,3.469523e+00
50%,6.792740e+01
75%,8.703458e+02
max,2.209663e+05


In [22]:
user = pd.DataFrame({"users":users["polygon"].keys(), "value": users["polygon"].values()})
user.describe()

,value
count,945.000000
mean,1.471958
std,3.464944
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,96.000000


In [68]:
# ngay nhieu liquidate
for timestamp, value in liquidate_data["polygon"].items():
    if value.get("number_wallets") > 30:
        print(f"{timestamp} - {value.get('number_wallets')}")


1689206400 - 33
1690675200 - 31
1692057600 - 41
1692144000 - 33
1692230400 - 286
1692662400 - 89
1694304000 - 44
1694390400 - 165
1695772800 - 45


In [26]:
# vi bi liquidate nhieu lan
for user, value in users["polygon"].items():
    if value > 5:
        if user not in remove_wallets["polygon"]:
            remove_wallets["polygon"].append(user)
        many_liquidated_users.append(user)
for timestamp in [1692230400, 1694390400]:
    remove_wallets["polygon"]+=liquidate_data["polygon"][timestamp]["wallets"]
remove_wallets["polygon"] = list(set(remove_wallets["polygon"]))

In [27]:
# sort debt token amount
m = chaintokens["polygon"][1692230400]['debt']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x7ceb23fd6bc0add59e62ac25578270cff1b9f619': 9.467469751491388,
 '0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270': 9.570001467692112,
 '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6': 1447.43010684,
 '0xa3fa99a148fa48d14ed51d610c367c61876997f1': 2038.2904540565085,
 '0xe111178a87a3bff0c8d18decba5798827539ae99': 18816.81627,
 '0xc2132d05d31c914a87c6611c10748aeb04b58e8f': 138002.321009,
 '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063': 281416.607332511,
 '0x2791bca1f2de4661ed88a30c99a7a9449aa84174': 867788.5782970003}

In [28]:
# sort debt token amount
m = tokens["polygon"][1692230400]['debt']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270': 1,
 '0x7ceb23fd6bc0add59e62ac25578270cff1b9f619': 2,
 '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6': 2,
 '0xa3fa99a148fa48d14ed51d610c367c61876997f1': 3,
 '0xe111178a87a3bff0c8d18decba5798827539ae99': 4,
 '0xc2132d05d31c914a87c6611c10748aeb04b58e8f': 68,
 '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063': 101,
 '0x2791bca1f2de4661ed88a30c99a7a9449aa84174': 173}

In [29]:
# sort debt token amount
m = chaintokens["polygon"][1692230400]['collateral']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x172370d5cd63279efa6d502dab29171933a610af': 12.9037100328762,
 '0x85955046df4668e1dd369d2de9f3aeb98dd2a369': 14.500335716969603,
 '0xfa68fb4628dff1028cfec22b4162fccd0d45efb6': 106.98091424632223,
 '0x0b3f868e0be5597d5db7feb59e1cadbb0fdda50a': 454.3157917055079,
 '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063': 816.0159501108612,
 '0x3a58a54c066fdc0f2d55fc9c89f0415c92ebf3c4': 4582.196239981902,
 '0xd6df932a45c0f255f85145f286ea0b292b21c90b': 11681.21906982145,
 '0x53e0bca35ec356bd5dddfebbd1fc0fd03fabad39': 26874.06250655153,
 '0x03b54a6e9a984069379fae1a4fc4dbae93b3bccd': 72607.90437590431,
 '0x2791bca1f2de4661ed88a30c99a7a9449aa84174': 94049.635322,
 '0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270': 215683.21787629317,
 '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6': 219076.90845660004,
 '0x7ceb23fd6bc0add59e62ac25578270cff1b9f619': 717733.3388100266}

In [30]:
# sort debt token amount
m = tokens["polygon"][1692230400]['collateral']
m = sorted(m.items(), key = lambda x: x[1])
dict(m)

{'0x85955046df4668e1dd369d2de9f3aeb98dd2a369': 1,
 '0x0b3f868e0be5597d5db7feb59e1cadbb0fdda50a': 1,
 '0xfa68fb4628dff1028cfec22b4162fccd0d45efb6': 4,
 '0x172370d5cd63279efa6d502dab29171933a610af': 4,
 '0x3a58a54c066fdc0f2d55fc9c89f0415c92ebf3c4': 5,
 '0x53e0bca35ec356bd5dddfebbd1fc0fd03fabad39': 6,
 '0xd6df932a45c0f255f85145f286ea0b292b21c90b': 7,
 '0x8f3cf7ad23cd3cadbd9735aff958023239c6a063': 7,
 '0x2791bca1f2de4661ed88a30c99a7a9449aa84174': 17,
 '0x03b54a6e9a984069379fae1a4fc4dbae93b3bccd': 27,
 '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6': 64,
 '0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270': 82,
 '0x7ceb23fd6bc0add59e62ac25578270cff1b9f619': 129}

## ARBITRUM

In [32]:
liquidate_analyze(liquidate_data["arbitrum"]).describe()

,timestamp,number_event,amount_in_usd,number_wallets
count,9.200000e+01,92.000000,92.000000,92.000000
mean,1.692101e+09,4.989130,15559.984539,4.586957
std,2.307058e+06,14.617838,104488.901620,12.848357
min,1.688170e+09,0.000000,0.000000,0.000000
25%,1.690135e+09,0.000000,0.000000,0.000000
50%,1.692101e+09,1.000000,64.219263,1.000000
75%,1.694066e+09,3.000000,1425.895589,3.000000
max,1.696032e+09,106.000000,996183.588352,86.000000


In [33]:
event = pd.DataFrame({"events":events["arbitrum"]})
event.describe()

,events
count,4.590000e+02
mean,3.118777e+03
std,1.389806e+04
min,9.406209e-09
25%,1.780489e+00
50%,4.216508e+01
75%,6.620441e+02
max,1.749634e+05


In [34]:
user = pd.DataFrame({"users":users["arbitrum"].keys(), "value": users["arbitrum"].values()})
user.describe()

,value
count,350.000000
mean,1.311429
std,1.079974
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,13.000000


In [69]:
# ngay nhieu liquidate
for timestamp, value in liquidate_data["arbitrum"].items():
    if value.get("number_wallets") > 30:
        print(f"{timestamp} - {value.get('number_wallets')}")


1692230400 - 86
1692316800 - 31
1692403200 - 68
1694390400 - 43
1695168000 - 31


In [38]:
# vi bi liquidate nhieu lan
for user, value in users["arbitrum"].items():
    if value > 5:
        if user not in remove_wallets["arbitrum"]:
            remove_wallets["arbitrum"].append(user)
        many_liquidated_users.append(user)
for timestamp in [1692230400, 1694390400]:
    remove_wallets["arbitrum"]+=liquidate_data["arbitrum"][timestamp]["wallets"]
remove_wallets["arbitrum"] = list(set(remove_wallets["arbitrum"]))

## OPTIMISM

In [39]:
liquidate_analyze(liquidate_data["optimism"]).describe()

,timestamp,number_event,amount_in_usd,number_wallets
count,9.200000e+01,92.000000,92.000000,92.000000
mean,1.692101e+09,1.782609,5787.309211,1.347826
std,2.307058e+06,6.946936,39742.532890,4.348638
min,1.688170e+09,0.000000,0.000000,0.000000
25%,1.690135e+09,0.000000,0.000000,0.000000
50%,1.692101e+09,0.000000,0.000000,0.000000
75%,1.694066e+09,1.000000,17.366483,1.000000
max,1.696032e+09,54.000000,375819.730800,28.000000


In [40]:
event = pd.DataFrame({"events":events["optimism"]})
event.describe()

,events
count,164.000000
mean,3246.539313
std,15548.870243
min,0.001780
25%,0.619097
50%,11.085021
75%,595.390390
max,168396.489340


In [41]:
user = pd.DataFrame({"events":users["optimism"]})
user.describe()

,events
count,99.000000
mean,1.656566
std,1.011784
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,5.000000


In [70]:
# ngay nhieu liquidate
for timestamp, value in liquidate_data["optimism"].items():
    if value.get("number_wallets") > 30:
        print(f"{timestamp} - {value.get('number_wallets')}")

In [45]:
# vi bi liquidate nhieu lan
for user, value in users["optimism"].items():
    if value > 5:
        if user not in remove_wallets["optimism"]:
            remove_wallets["optimism"].append(user)
    
        many_liquidated_users.append(user)
for timestamp in [1692230400, 1694390400]:
    remove_wallets["optimism"]+=liquidate_data["optimism"][timestamp]["wallets"]
remove_wallets["optimism"] = list(set(remove_wallets["optimism"]))

## AVALANCHE

In [47]:
liquidate_analyze(liquidate_data["avalanche"]).describe()

,timestamp,number_event,amount_in_usd,number_wallets
count,9.200000e+01,92.000000,9.200000e+01,92.000000
mean,1.692101e+09,3.934783,2.068832e+04,3.402174
std,2.307058e+06,10.072053,1.759630e+05,8.083438
min,1.688170e+09,0.000000,0.000000e+00,0.000000
25%,1.690135e+09,0.000000,0.000000e+00,0.000000
50%,1.692101e+09,1.000000,3.126690e+01,1.000000
75%,1.694066e+09,3.000000,5.186684e+02,2.250000
max,1.696032e+09,63.000000,1.687327e+06,47.000000


In [48]:
event = pd.DataFrame({"events":events["avalanche"]})
event.describe()

,events
count,3.620000e+02
mean,5.257806e+03
std,7.965669e+04
min,2.375876e-02
25%,2.540242e+01
50%,1.120222e+02
75%,5.773886e+02
max,1.515093e+06


In [49]:
user = pd.DataFrame({"events":users["avalanche"]})
user.describe()

,events
count,248.000000
mean,1.459677
std,1.619213
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,20.000000


In [71]:
# ngay nhieu liquidate
for timestamp, value in liquidate_data["avalanche"].items():
    if value.get("number_wallets") > 30:
        print(f"{timestamp} - {value.get('number_wallets')}")


1692230400 - 36
1692662400 - 40
1694390400 - 47


In [54]:
# vi bi liquidate nhieu lan
for user, value in users["avalanche"].items():
    if value > 5:
        many_liquidated_users.append(user)
        if user not in remove_wallets["avalanche"]:
            remove_wallets["avalanche"].append(user)
for timestamp in [1692230400, 1694390400]:
    remove_wallets["avalanche"]+=liquidate_data["avalanche"][timestamp]["wallets"]
remove_wallets["avalanche"] = list(set(remove_wallets["avalanche"]))

In [73]:
m = []
for key in remove_wallets:
    print(f"{key} - ", len(remove_wallets[key]))
    m += remove_wallets[key]
print(len(list(set(m))))

bsc -  0
ftm -  0
ethereum -  322
polygon -  1346
arbitrum -  478
optimism -  80
avalanche -  204
1483


In [74]:
import json
with open("data/debtors.json", 'r') as f:
    debtors = json.loads(f.read())
result = []
for i in debtors:
    if i not in m:
        result.append(i)

In [75]:
with open("user.json", 'w') as f:
    json.dump(list(set(result)), f, indent=2)

In [66]:
for _id, value in Chain.mapping.items():
    data = {"_id": f"wallet_flags_{value}", "newElite": {i: True for i in result}}
    with open(f"{_id}.json", 'w') as f:
        json.dump(data, f, indent=2)